# Note:
- This notebook file may contain methods or algorithms that are NOT covered by the teaching content of BT4222 and hence will not be assessed in your midterm exam.
- It serves to increase your exposure in depth and breath to the practical methods in addressing the specific project topic. We believe it will be helpful for your current project and also your future internship endeavors.

# **Import Library**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# **Define Network Structure**
The network structure was from "Performance Analysis of Different Neural Networks for Sentiment Analysis on IMDb Movie Reviews" Figure3. and we add two more FC layers for classification.
```
conv1.weight 	 torch.Size([32, 1, 3])
conv1.bias 	 torch.Size([32])
Bn1.weight 	 torch.Size([32])
Bn1.bias 	 torch.Size([32])
Bn1.running_mean 	 torch.Size([32])
Bn1.running_var 	 torch.Size([32])
Bn1.num_batches_tracked 	 torch.Size([])
bi_lstm1.weight_ih_l0 	 torch.Size([512, 50])
bi_lstm1.weight_hh_l0 	 torch.Size([512, 128])
bi_lstm1.bias_ih_l0 	 torch.Size([512])
bi_lstm1.bias_hh_l0 	 torch.Size([512])
bi_lstm2.weight_ih_l0 	 torch.Size([512, 128])
bi_lstm2.weight_hh_l0 	 torch.Size([512, 128])
bi_lstm2.bias_ih_l0 	 torch.Size([512])
bi_lstm2.bias_hh_l0 	 torch.Size([512])
fc1.weight 	 torch.Size([100, 1024])
fc1.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([5, 100])
fc2.bias 	 torch.Size([5])
```



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, 3, 1,1, bias=True)
        self.Bn1 = nn.BatchNorm1d(32)
        #########################################

        self.bi_lstm1 = nn.LSTM(input_size=50, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)
        self.bi_lstm2 = nn.LSTM(input_size=128, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)

        self.conv1 = nn.Conv1d(1, 32, 3, 1,1, bias=True)
        self.Bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.AvgPool1d(kernel_size=4, stride=4)

        self.fc1 = nn.Linear(1024, 100, bias=True)
        self.fc2 = nn.Linear(100, 5, bias=True)



    def forward(self, x):
        x = torch.flatten(x, 1)
        x, _ = self.bi_lstm1(x)
        x, _ = self.bi_lstm2(x)
        x = x.view(-1, 1, 128)
        x = F.relu(self.Bn1(self.conv1(x)))
        x = self.pool1(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x



# **Training and Testing**

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()  # Set the model to training mode

    for batch_idx, (data, target) in enumerate(train_loader):  # Loop over each batch from the training set
        data, target = data.to(device), target.to(device)  # Move the data to the device that is used

        target = target-1  # Adjust the target values (Moving 1-5 to 0-4  for easy training)
        target = target.long()  # Make sure that target data is long type (necessary for loss function)

        optimizer.zero_grad()  # Clear gradients from the previous training step
        output = model(data)  # Run forward pass (model predictions)

        loss = F.cross_entropy(output, target)  # Calculate the loss between the output and target
        loss.backward()  # Perform backpropagation (calculate gradients of loss w.r.t. parameters)
        optimizer.step()  # Update the model parameters

        if batch_idx % args.log_interval == 0:  # Print log info for specified interval
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0

    with torch.no_grad():  # Deactivates autograd, reduces memory usage and speeds up computations
        for data, target in test_loader:  # Loop over each batch from the testing set
            data, target = data.to(device), target.to(device)  # Move the data to the device that is used
            target = target-1  # Adjust the target values
            output = model(data)  # Run forward pass (model predictions)
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability as the predicted output
            correct += pred.eq(target.view_as(pred)).sum().item()  # Count correct predictions

    test_loss /= len(test_loader.dataset)  # Calculate the average loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
    return correct  # Return the number of correctly classified samples


# **Hyperparameter**

We use only cpu here as an example. learning rate is set as 1.

In [ ]:
class Args:
  epochs = 10
  lr = 1.0
  use_cuda=False
  gamma = 0.7
  log_interval = 10
  no_cuda = False
  seed = 1

args = Args()

# **Load Data**

In [ ]:
from google.colab import drive

import gdown

file_id = '1CCIfElCaURQbuYvHZiL445UQIRzmmuM7'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'train_vectors.pt'
gdown.download(url, output, quiet=False)

file_id = '1bwkg7XdmH6Mkp_tkAakCbxJMWNAXJU43'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'train_labels.pt'
gdown.download(url, output, quiet=False)

file_id = '1fprUkqC9Qb-y1eDRZt0gA4-4gS941TUo'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'test_vectors.pt'
gdown.download(url, output, quiet=False)

file_id = '1VwOqpW7DZPhqAGDrreVwhtzCB2lUc_LD'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'test_labels.pt'
gdown.download(url, output, quiet=False)

train_vectors = torch.load('train_vectors.pt')
train_labels = torch.load('train_labels.pt')
test_vectors = torch.load('test_vectors.pt')
test_labels = torch.load('test_labels.pt')





Downloading...
From: https://drive.google.com/uc?id=1CCIfElCaURQbuYvHZiL445UQIRzmmuM7
To: /content/train_vectors.pt
100%|██████████| 80.0M/80.0M [00:00<00:00, 127MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bwkg7XdmH6Mkp_tkAakCbxJMWNAXJU43
To: /content/train_labels.pt
100%|██████████| 3.20M/3.20M [00:00<00:00, 157MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fprUkqC9Qb-y1eDRZt0gA4-4gS941TUo
To: /content/test_vectors.pt
100%|██████████| 20.0M/20.0M [00:00<00:00, 134MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1VwOqpW7DZPhqAGDrreVwhtzCB2lUc_LD
To: /content/test_labels.pt
100%|██████████| 801k/801k [00:00<00:00, 91.4MB/s]


# **Start training and testing**

In [ ]:
torch.manual_seed(args.seed)

device = torch.device("cuda" if args.use_cuda else "cpu")
model = Net().to(device)

for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())

#Form training and testing dataset
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

train_dataset = torch.utils.data.TensorDataset(train_vectors, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_vectors, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=640, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=640, shuffle=False)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

#Model training
ACC = 0
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    ACC_ = test(model, device, test_loader)
    if ACC_>ACC or ACC_ == ACC:
        ACC = ACC_
        torch.save(model.state_dict(), "Baseline_CNN_lstm.pt")

    scheduler.step()

print(ACC)


conv1.weight 	 torch.Size([32, 1, 3])
conv1.bias 	 torch.Size([32])
Bn1.weight 	 torch.Size([32])
Bn1.bias 	 torch.Size([32])
Bn1.running_mean 	 torch.Size([32])
Bn1.running_var 	 torch.Size([32])
Bn1.num_batches_tracked 	 torch.Size([])
bi_lstm1.weight_ih_l0 	 torch.Size([512, 50])
bi_lstm1.weight_hh_l0 	 torch.Size([512, 128])
bi_lstm1.bias_ih_l0 	 torch.Size([512])
bi_lstm1.bias_hh_l0 	 torch.Size([512])
bi_lstm2.weight_ih_l0 	 torch.Size([512, 128])
bi_lstm2.weight_hh_l0 	 torch.Size([512, 128])
bi_lstm2.bias_ih_l0 	 torch.Size([512])
bi_lstm2.bias_hh_l0 	 torch.Size([512])
fc1.weight 	 torch.Size([100, 1024])
fc1.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([5, 100])
fc2.bias 	 torch.Size([5])
Train Epoch: 1 [0/400000 (0%)]	Loss: 1.611336
Train Epoch: 1 [6400/400000 (2%)]	Loss: 1.649561
Train Epoch: 1 [12800/400000 (3%)]	Loss: 1.567207
Train Epoch: 1 [19200/400000 (5%)]	Loss: 1.541761
Train Epoch: 1 [25600/400000 (6%)]	Loss: 1.450460
Train Epoch: 1 [32000/400000 (8%)]	Loss: 1.